In [129]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv

In [138]:


def get_soup(url):
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful
    return BeautifulSoup(response.text, 'html.parser')

def get_restaurant_name_and_review_count(soup):
    # Replace 'restaurant-name-class' and 'review-count-class' with the actual classes
    restaurant_name = soup.select_one('.css-1se8maq').text.strip()
    review_count = soup.select_one('.css-19v1rkv').text.strip()
    return restaurant_name, review_count

def get_reviews(soup):
    # Replace 'review-container-class' with the actual class
    review_elements = soup.select('css-19v1rkv')
    reviews = []
    for review_element in review_elements:
        # Replace 'review-text-class', 'reviewer-class', 'rating-class' with the actual classes
        review_text = review_element.select_one('comment__373c0__1Mf84').text.strip()
        reviewer = review_element.select_one('link__09f24__1kwXV').text.strip()
        rating = review_element.select_one('i-stars__09f24__1T6r')['title']  # Assuming rating is in the title attribute
        reviews.append((review_text, reviewer, rating))
    return reviews

def save_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Review Text', 'Reviewer', 'Rating'])  # Header
        writer.writerows(data)

def main(url):
    soup = get_soup(url)
    restaurant_name, review_count = get_restaurant_name_and_review_count(soup)
    reviews = get_reviews(soup)
    
    print(f'Restaurant Name: {restaurant_name}')
    print(f'Review Count: {review_count}')
    print(f'Reviews: {reviews}')
    
    save_to_csv(reviews, 'reviews.csv')

# Replace with the actual URL
url = 'https://www.yelp.ca/biz/pai-northern-thai-kitchen-toronto-5?osq=Restaurants'
main(url)

Restaurant Name: Pai Northern Thai Kitchen
Review Count: (3,497 reviews)
Reviews: []


In [144]:

def scrape_yelp_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
   
    # Extracting the restaurant name with error handling
    restaurant_name_tag = soup.find('h1', class_='css-11q1g5y')
    restaurant_name = restaurant_name_tag.text if restaurant_name_tag else "N/A"

    # Extracting the total number of reviews with error handling
    total_reviews_tag = soup.find('span', class_='text__09f24__2tZKC')
    total_reviews = total_reviews_tag.text.split()[0] if total_reviews_tag else "N/A"

    # Extracting the reviews details
    review_elements = soup.select('.review__373c0__13kpL')
    reviews = []
    reviewers = []
    ratings = []
    
    for review_element in review_elements:
        review_text = review_element.select_one('p', class_='comment__373c0__1Mf84').text
        reviewer = review_element.select_one('a', class_='link__09f24__1kwXV').text
        rating_element = review_element.select_one('div', class_='i-stars__09f24__1T6rz')
    
        if rating_element:
            aria_label = rating_element['aria-label']
            rating_value = aria_label.split(' ')[0]
        else:
            print("Rating Value not found")
            rating_value = "N/A"
    
        reviews.append(review_text)
        reviewers.append(reviewer)
        ratings.append(rating_value)
        
    review_data = {
        "Reviewer": reviewers,
        "Review": reviews,
        "Rating": ratings,
    }
    
    return restaurant_name, total_reviews, review_data

def save_to_csv(restaurant_name, total_reviews, review_data):
    df = pd.DataFrame(review_data)
    df.to_csv('yelp_reviews.csv', index=False)

if __name__ == '__main__':
    url = "https://www.yelp.ca/biz/pai-northern-thai-kitchen-toronto-5?osq=Restaurants"
    restaurant_name, total_reviews, reviews = scrape_yelp_data(url)  # Corrected variable names
    save_to_csv(restaurant_name, total_reviews, reviews)
    print("Data saved to yelp_reviews.csv")

Data saved to yelp_reviews.csv


In [151]:
final = pd.read_csv("yelp_reviews.csv")

In [153]:
final

,Reviewer,Review,Rating


In [19]:
import requests
from bs4 import BeautifulSoup
import csv

def scrapingrestaurantinfo(url_list, output_csv):
    reviews = []

    for url in url_list:
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            restaurant_name = soup.find('h1', class_='css-1se8maq').text.strip()
            total_reviews = soup.find('a', class_='css-19v1rkv').text.strip()

            review_elements = soup.find_all('li', class_='css-1q2nwpv')
            for review_element in review_elements:
                review_text = review_element.find('span', {'lang': 'en'}).text.strip()
                reviewer = review_element.find('a', class_='css-19v1rkv').text.strip()
                
                # Handle missing rating element gracefully
                rating_element = review_element.find('div', class_='five-stars__09f24__mBKym five-stars--regular__09f24__DgBNj css-1jq1ouh')
                rating = rating_element['aria-label'].split()[0] if rating_element else "N/A"
                
                reviews.append([restaurant_name, total_reviews, review_text, reviewer, rating])
        
        else:
            print(f'Failed to retrieve data from {url}')

    with open(output_csv, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Restaurant Name', 'Total Reviews', 'Review Text', 'Reviewer', 'Rating'])
        writer.writerows(reviews)
        print(f'I have scraped information from the webpage and stored information in {output_csv}')
    
output_csv_file = '101501825_yelp_reviews.csv'

restaurant_list = ['https://www.yelp.ca/biz/pai-northern-thai-kitchen-toronto-5?osq=Restaurants',
                   'https://www.yelp.ca/biz/ramen-misoya-toronto-toronto-2?osq=Restaurants',
                   'https://www.yelp.ca/biz/hana-don-toronto-2?osq=Restaurants']

scrapingrestaurantinfo(restaurant_list, output_csv_file)


I have scraped information from the webpage and stored information in 101501825_yelp_reviews.csv


In [21]:
final = pd.read_csv("101501825_yelp_reviews.csv")

In [22]:
final

,Restaurant Name,Total Reviews,Review Text,Reviewer,Rating
0,Pai Northern Thai Kitchen,"(3,568 reviews)",My friend and I wanted to catch up over dinner...,Maria J.,NaN
1,Pai Northern Thai Kitchen,"(3,568 reviews)",- excellent chips!- pad thai was spicy (level:...,Albert K.,NaN
2,Pai Northern Thai Kitchen,"(3,568 reviews)","My friend loves this restaurant. Me, well, it'...",Becky G.,NaN
3,Pai Northern Thai Kitchen,"(3,568 reviews)",Seeking a restaurant with all the vibes?!++ CO...,Carly W.,NaN
4,Pai Northern Thai Kitchen,"(3,568 reviews)",Friday night dinner. Tried to book 2 weeks in ...,Aiko I.,NaN
5,Pai Northern Thai Kitchen,"(3,568 reviews)",We went here on a Thursday to get in line arou...,Elaine N.,NaN
6,Pai Northern Thai Kitchen,"(3,568 reviews)","Hubby and I saw this place on YouTube's ""Worth...",Brenda C.,NaN
7,Pai Northern Thai Kitchen,"(3,568 reviews)",The place was decorated really well with a tro...,Sandra K.,NaN
8,Pai Northern Thai Kitchen,"(3,568 reviews)",I was in Toronto attending the annual TIFF fil...,Louisa M.,NaN
9,Pai Northern Thai Kitchen,"(3,568 reviews)",Pai is one of the most popular go-to places lo...,Cindy N.,NaN
